# [Polars](https://www.pola.rs/) Rust Cheat Sheet

[![Polars cheat sheet: download as PDF](https://github.com/akashsoni01/cargo_project/blob/main/jupiter/polars%20cheat%20sheet.pdf)

Most examples were taken from the official [Polars user guide](https://pola-rs.github.io/polars-book/user-guide/).

By Akash Soni ([Github](https://github.com/akashsoni01)), inspired by [Pandas Cheat Sheet](https://franzdiebold.github.io/polars-cheat-sheet/Polars_cheat_sheet.pdf/).

#### Add dependency for polars

In [ ]:
:dep polars = { version = "0.36.2", features = ["lazy", "temporal", "describe", "json", "parquet", "dtype-datetime"] }
:dep chrono = "0.4.31"

#### import the package 

In [19]:
use chrono::prelude::*;
use std::fs::File;


##### Create a data frame using few ints, dates, floats 

In [23]:
use std::fs::File;

use chrono::prelude::*;

let mut df: DataFrame = df!(
    "integer" => &[1, 2, 3],
    "date" => &[
            NaiveDate::from_ymd_opt(2022, 1, 1).unwrap().and_hms_opt(0, 0, 0).unwrap(),
            NaiveDate::from_ymd_opt(2022, 1, 2).unwrap().and_hms_opt(0, 0, 0).unwrap(),
            NaiveDate::from_ymd_opt(2022, 1, 3).unwrap().and_hms_opt(0, 0, 0).unwrap(),
    ],
    "float" => &[4.0, 5.0, 6.0]
)
.unwrap();
println!("{}", df);


shape: (3, 3)
┌─────────┬─────────────────────┬───────┐
│ integer ┆ date                ┆ float │
│ ---     ┆ ---                 ┆ ---   │
│ i32     ┆ datetime[ms]        ┆ f64   │
╞═════════╪═════════════════════╪═══════╡
│ 1       ┆ 2022-01-01 00:00:00 ┆ 4.0   │
│ 2       ┆ 2022-01-02 00:00:00 ┆ 5.0   │
│ 3       ┆ 2022-01-03 00:00:00 ┆ 6.0   │
└─────────┴─────────────────────┴───────┘


##### Reading and writing data frame to a file 

Make sure you have already created folder directory doc/data if not create it in your project repo. 

/*
example in python

df.write_csv("docs/data/output.csv")
df_csv = pl.read_csv("docs/data/output.csv")
print(df_csv)

*/

In [24]:
let mut file = File::create("docs/data/output.csv").expect("could not create file");
CsvWriter::new(&mut file)
    .include_header(true)
    .with_separator(b',')
    .finish(&mut df)?;


let df_csv = CsvReader::from_path("docs/data/output.csv")?
    .infer_schema(None)
    .has_header(true)
    .finish()?;
println!("{}", df_csv);


shape: (3, 3)
┌─────────┬─────────────────────────┬───────┐
│ integer ┆ date                    ┆ float │
│ ---     ┆ ---                     ┆ ---   │
│ i64     ┆ str                     ┆ f64   │
╞═════════╪═════════════════════════╪═══════╡
│ 1       ┆ 2022-01-01T00:00:00.000 ┆ 4.0   │
│ 2       ┆ 2022-01-02T00:00:00.000 ┆ 5.0   │
│ 3       ┆ 2022-01-03T00:00:00.000 ┆ 6.0   │
└─────────┴─────────────────────────┴───────┘


We can clearly see the code .with_try_parse_dates(true) is converting data in data that is of type string to date while reading file. 

In [25]:
let df_csv = CsvReader::from_path("docs/data/output.csv")?
    .infer_schema(None)
    .has_header(true)
    .with_try_parse_dates(true)
    .finish()?;
println!("{}", df_csv);


shape: (3, 3)
┌─────────┬─────────────────────┬───────┐
│ integer ┆ date                ┆ float │
│ ---     ┆ ---                 ┆ ---   │
│ i64     ┆ datetime[μs]        ┆ f64   │
╞═════════╪═════════════════════╪═══════╡
│ 1       ┆ 2022-01-01 00:00:00 ┆ 4.0   │
│ 2       ┆ 2022-01-02 00:00:00 ┆ 5.0   │
│ 3       ┆ 2022-01-03 00:00:00 ┆ 6.0   │
└─────────┴─────────────────────┴───────┘


##### Create and Read JSON file 

In [29]:
let mut file = File::create("docs/data/output.json").expect("could not create file");
JsonWriter::new(&mut file).finish(&mut df)?;
let f = File::open("docs/data/output.json")?;
let df_json = JsonReader::new(f)
    .with_json_format(JsonFormat::JsonLines)
    .finish()?;
println!("{}", df_json);


shape: (3, 3)
┌─────────┬─────────────────────┬───────┐
│ integer ┆ date                ┆ float │
│ ---     ┆ ---                 ┆ ---   │
│ i64     ┆ str                 ┆ f64   │
╞═════════╪═════════════════════╪═══════╡
│ 1       ┆ 2022-01-01 00:00:00 ┆ 4.0   │
│ 2       ┆ 2022-01-02 00:00:00 ┆ 5.0   │
│ 3       ┆ 2022-01-03 00:00:00 ┆ 6.0   │
└─────────┴─────────────────────┴───────┘
